<a href="https://colab.research.google.com/github/choi-yongsuk/deep-learning-nlp/blob/master/Self_supervised_learning_%26_Fine_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 실습자료:
# https://tinyurl.com/SKT20220818A

##**HuggingFace 라이브러리 사용하기**
지난 시간에는 Huggingface의 Tokenizers 라이브러리를 사용해보았습니다. 이번 시간에는 사전학습 모델들이 포함되어 있는 Transformers 라이브러리와 자연어처리 데이터셋들을 받을 수 있는 datasets 라이브러리를 사용해보겠습니다.

### **필요 패키지 다운로드 및 import**

In [ ]:
!pip install transformers[sentencepiece]
!pip install datasets

In [ ]:
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
import time

# 데이터 크롤링을 위한 라이브러리
import requests
from bs4 import BeautifulSoup

# Pytorch
import torch
import torch.nn as nn

# Huggingface
import transformers
import datasets

### **Huggingface's Transformers**

- 시작하기 전에 Huggingface에서 제공하는 Transformers에 대하여 알아봅니다. 
- 자연어 처리 관련 여러 라이브러리가 있지만 Transformer를 활용한 자연어 처리 task에서 가장 많이 활용되고 있는 라이브러리는 transformers입니다.
- pytorch version의 BERT를 가장 먼저 구현하며 주목받았던 huggingface는 현재 transformer기반의 다양한 모델들은 구현 및 공개하며 많은 주목을 받고 있습니다.([Pre-trained Transformers](https://huggingface.co/models))
- tensorflow, pytorch 버전의 모델 모두 공개되어 있어 다양한 상황에서 활용하기 좋습니다.
- 등록된 모델 이외에도 custom model을 업로드하여 사용할 수 있습니다.
- Transformers Documentation과 실습 자료를 이용해 transformers 라이브러리에 대해 알아봅니다.
- [Transformers Library](https://huggingface.co/transformers/)

#### Main Classes
- Configuration: https://huggingface.co/transformers/main_classes/configuration.html
- AutoConfig에서는 다양한 모델의 configuration (환경 설정)을 string tag를 이용해 쉽게 load할 수 있습니다.
- 각 Config에는 해당 모델 architecture와 task에 필요한 다양한 정보(architecture 종류, 레이어 수, hidden unit size, hyperparameter)를 담고 있습니다.
- [Pre-trained Transformers](https://huggingface.co/models)에서 해당 모델들의 name tag를 확인할 수 있습니다.

In [ ]:
from transformers import AutoConfig

# AutoConfig을 이용해 "BERT" 모델의 configuration을 받아봅시다.
config = AutoConfig.from_pretrained('bert-large-uncased')
config

In [ ]:
# AutoConfig을 이용해 "GPT-2" 모델의 configuration을 받아봅시다.
gpt_config = AutoConfig.from_pretrained('gpt2')

In [ ]:
gpt_config

In [ ]:
print(config.vocab_size)

In [ ]:
# config 을 편하게 사용하기 위해 .to_dict() 함수를 통해 dict형식으로 바꿀 수 있습니다.
config_dict = config.to_dict()
config_dict

In [ ]:
# AutoConfig을 사용하지 않고, 특정 모델의 config임을 명시하여 사용할 수도 있습니다.

from transformers import BertConfig
bertconfig = BertConfig.from_pretrained('bert-base-uncased')

In [ ]:
# 이렇게 명시하여 사용하는 경우, 다른 모델의 config에 있는 값들을 원하는 모델의 config 형식으로 변환하여 사용할수도 있습니다.
bert_in_gpt2_config = BertConfig.from_pretrained('gpt2')

In [ ]:
bert_in_gpt2_config

- Model: https://github.com/huggingface/transformers/tree/master/src/transformers/models
    - Transformers에서는 transformer기반의 모델 architecture를 구현해두었습니다.
    - 최근에는 [ViT](https://arxiv.org/abs/2010.11929)와 같이 Vision task에서 활용하는 transformer 모델들을 추가하며 그 확장성을 더해가고 있습니다.
    - 모델 architecture 뿐만 아니라 관련 task에 적용가능한 형태의 구현체들이 있습니다.
    - BERT 구현체에서 제공하고 있는 class를 확인하고 해당 구조를 이용해 학습한 모델들을 load해보겠습니다.


In [ ]:
from transformers import BertForMaskedLM, BertForQuestionAnswering, BertForSequenceClassification, BertForTokenClassification, BertForMultipleChoice, BertModel

In [ ]:
from transformers import AutoModel, AutoTokenizer, AutoConfig

# BERT model
![image](https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2Fb3a7WV%2FbtqVZTeLXwY%2FmDrKNb2oGLzUJPW5N7Azlk%2Fimg.png)

- BERT 모델은 transformer 모델의 "Encoder" 부분만 사용한 형태의 모델입니다.
![](https://pytorch.org/tutorials/_images/transformer_architecture.jpg)

Huggingface library를 이용해 직접 BERT 모델을 불러와보도록 하겠습니다.

In [ ]:
bertmodel = AutoModel.from_pretrained('bert-base-uncased')

BERT 모델의 레이어들을 확인해보시면, 아래와 같은 레이어들로 구성된 것을 알 수 있습니다. 

- word_embeddings, position_mebeddings (Transformer의 word embedding과 positionl encoding)
- token_type_embeddings (BERT에 새롭게 추가된 입력 문장의 인덱스 임베딩)
- BertLayer
  - attention (multi-head attention)
  - intermediate + output (FeedFoward)

이는 Transformer encoder에 token type embedding만 추가된 형태입니다.


In [ ]:
bertmodel

모델의 word embedding을 사용하기 위해서는, 우선 입력 문장을 word (혹은 sub-word)들로 나누어 index로 변환해주는 작업이 필요하겠죠? 우리는 이런 일을 해주는 것을 tokenizer라고 부릅니다.

BERT 모델이 사용하는 tokenizer도 불러와봅시다.

BERT tokenizer는 문장이 입력되면, 
- 여러개의 token들로 쪼개주고,
- 쪼개진 token들을 index로 변환해주고,
- attention에 사용되는 mask, token type(문장 인덱스), 추가로 필요한 토큰들 (CLS, SEP)까지 알아서 추가해줍니다.

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
input = tokenizer('Hello, world. The dog is so cute. I love you.')
input

특정 태스크에 특화된 형태의 모델을 불러올 수도 있습니다.

- BertForQuestionAnswering의 경우 마지막 레이어의 output dimension이 2인 것을 확인할 수 있습니다.
- 단순히 "bert-base-uncased"를 이용해 불러오면, BERT pre-training이 완료된 모델을 사용하지만, "deepset/bert-base-cased-squad2"에서 모델을 불러오면 SQuAD 2.0 데이터셋에 fine-tuning 까지 완료된 모델을 불러올수도 있습니다.

In [ ]:
bert_qa = BertForQuestionAnswering.from_pretrained('bert-base-uncased')

In [ ]:
bert_qa

In [ ]:
bert_qa = BertForQuestionAnswering.from_pretrained('deepset/bert-base-cased-squad2') # SQuAD 2.0 데이터셋에 fine-tuning까지 완료된 모델입니다.

In [ ]:
input.keys()

In [ ]:
input = tokenizer('Hello, world. The dog is so cute. I love you.', return_tensors='pt')

input['input_ids'], input['input_ids'].shape

In [ ]:
bert_qa(**input)['start_logits']

In [ ]:
bert_qa(**input)['start_logits'].shape

Token Classification 모델의 경우, 입력된 각 단어마다의 classification 이 필요한 경우에 사용할 수 있는 모델입니다. (ex. Named Entity Recognition)

In [ ]:
bert_token_cls = BertForTokenClassification.from_pretrained('ckiplab/bert-base-chinese-ner')

In [ ]:
input = tokenizer('Hello, world. The dog is so cute. I love you.', return_tensors='pt')
bert_token_cls(**input)

# Optimization
https://huggingface.co/transformers/main_classes/optimizer_schedules.html
- optimization에서는 널리 쓰이고 있는 다양한 optimizer를 제공하고 있습니다.
- 이와 관련된 learning rate을 조절하는 scheduler도 제공하고 있습니다.
- 물론, PyTorch 라이브러리에서 제공하는 것을 사용해도 됩니다.

In [ ]:
from transformers import AdamW, get_linear_schedule_with_warmup

In [ ]:
bert_maskedlm = BertForMaskedLM.from_pretrained('bert-base-uncased')

parameters = bert_maskedlm.parameters()
# parameters = bert_maskedlm.named_parameters()
optimizer = AdamW(parameters, lr=5e-5)
total_training_step = 100
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=int(total_training_step/10), num_training_steps=total_training_step)

# loss.backward()
optimizer.step()
scheduler.step()

# Pipelines
https://huggingface.co/transformers/main_classes/pipelines.html
- pipeline에서는 단 1줄로 모델을 load할 수 있습니다.
- 현재 pipeline에서 제공하고 있는 task들은 다음과 같습니다.
    - fill-mask
    - text-classification
    - text-generation
    - sentiment-analysis
    - text2text-generation
    - ner
    - translation_xx_to_yy
    - zero-shot-classification
- 예시를 통해 pipeline class를 어떻게 활용하는지 알아보겠습니다.

In [ ]:
from transformers import pipeline, AutoTokenizer, AutoModel

In [ ]:
fill_masker = pipeline("fill-mask")
fill_masker("I <mask> you.")

In [ ]:
classifier = pipeline("text-classification")
classifier("This restaurant is not bad.")

In [ ]:
en_fr_translator = pipeline("translation_en_to_fr")
en_fr_translator("How old are you?")
# Quel âge avez-vous?

In [ ]:
en_fr_translator("What is your name?")

# Tokenizer
https://huggingface.co/transformers/main_classes/tokenizer.html
- tokenizer에서는 tokenization과 관련된 다양한 기능을 제공하고 있습니다.
- string을 tokenization, token을 string으로 바꿔주는 기능은 input embedding을 만들거나 model의 output을 decoding하기 위해 사용됩니다.
- tokenizer에서는 주어진 tokenization config를 바탕으로 transformer input으로 필요한 정보를 생성합니다.

Q. Pretrained model만이 아니라, tokenizer도 pretrained 된 것을 사용하는 이유가 무엇일까요?

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

In [ ]:
tokenizer.get_vocab()

In [ ]:
print(tokenizer.tokenize("I love natural language processing"))
print(tokenizer.convert_tokens_to_ids(tokenizer.tokenize("I love natural language processing")))
print(tokenizer.convert_tokens_to_string(tokenizer.tokenize("I love natural language processing")))

In [ ]:
tokenizer("I love natural language processing")

## BERT를 이용한 네이버 영화 리뷰 sentiment classification

BERT모델을 fine-tuning하여 네이버 영화리뷰가 긍정적인 리뷰인지, 아닌지를 분석하는 실험을 진행해보겠습니다.

이번 실습에서는 실제 상황과 비슷하게, 네이버 영화리뷰 데이터를 크롤링하여 데이터를 생성해보겠습니다.

In [ ]:
# 1 page 데이터를 읽어와보겠습니다.
page = 1
url = f'https://movie.naver.com/movie/point/af/list.naver?&page={page}'
url


In [ ]:
# url로부터 html정보를 얻어옵니다.
html = requests.get(url)
#html을 받아온 문서를 .content로 지정 후 soup객체로 변환
soup = BeautifulSoup(html.content,'html.parser')
soup

In [ ]:
#find_all : 지정한 태그의 내용을 모두 찾아 리스트로 반환
reviews = soup.find_all("td",{"class":"title"})
reviews

In [ ]:
review_data = []
#한 페이지의 리뷰 리스트의 리뷰를 하나씩 보면서 데이터 추출
for review in reviews:
    # 예시: <a class="report" href="#" onclick="report('nimo****', 'VYvWCsRvbPwqbJZjcJx6mHaPm5IRnI1v1MYlRfqlIlA=', '재밌는데 겟아웃 보단…', '18375693', 'point_after');" style="color:#8F8F8F" title="새 창">신고</a>
    sentence = review.find("a",{"class":"report"}).get("onclick").split("', '")[2]
    #만약 리뷰 내용이 비어있다면 데이터를 사용하지 않음
    if sentence != "":
        # 예시: <a class="movie color_b" href="?st=mcode&amp;sword=194196&amp;target=after">한산: 용의 출현</a>
        movie = review.find("a",{"class":"movie color_b"}).get_text()
        # 예시:  <span class="st_off"><span class="st_on" style="width:100%">별점 - 총 10점 중</span></span><em>10</em>
        score = review.find("em").get_text()
        review_data.append([movie,sentence,int(score)])

review_data

In [ ]:
def get_page_reviews(page):
    review_data = []
    url = f'https://movie.naver.com/movie/point/af/list.naver?&page={page}'
    #get : request로 url의  html문서의 내용 요청
    html = requests.get(url)
    #html을 받아온 문서를 .content로 지정 후 soup객체로 변환
    soup = BeautifulSoup(html.content,'html.parser')
    #find_all : 지정한 태그의 내용을 모두 찾아 리스트로 반환
    reviews = soup.find_all("td",{"class":"title"})
    
    #한 페이지의 리뷰 리스트의 리뷰를 하나씩 보면서 데이터 추출
    for review in reviews:
        sentence = review.find("a",{"class":"report"}).get("onclick").split("', '")[2]
        #만약 리뷰 내용이 비어있다면 데이터를 사용하지 않음
        if sentence != "":
            movie = review.find("a",{"class":"movie color_b"}).get_text()
            score = review.find("em").get_text()
            review_data.append([movie,sentence,int(score)])
    return review_data


from tqdm.contrib.concurrent import process_map # 병렬처리+진행도 시각화를 위한 라이브러리입니다.
review_data = process_map(get_page_reviews, range(1, 1000), max_workers=8, chunksize=8) # multiprocessing을 통해 데이터를 크롤링합니다.

In [ ]:
review_data[:10]

In [ ]:
df = pd.DataFrame(sum(review_data, []))
df.columns = ['movie','review','score']
df = df.dropna()
df

### **Training Movie Review Classifier with BERTForSequenceClassification Class**

Pre-trained BERT의 config, tokenizer, model을 각각 불러오겠습니다.

이 때, 한국어에 특화되어 학습된 모델 중 kcBERT를 이용해보겠습니다.
- kcBERT 모델은 네이버 뉴스의 댓글과 대댓글을 이용하여 사전학습한 모델로, 기존의 koBERT 모델들이 정형화된 언어만 사용하는 것에 비해 구어체와 신조어 등을 더 많이 학습한 모델입니다. 네이버 영화 리뷰 역시 구어체와 신조어가 많은 데이터이므로, kcBERT가 적합한 모델입니다.

- https://github.com/Beomi/KcBERT

- https://huggingface.co/beomi/kcbert-base
<!-- https://huggingface.co/monologg/kobert -->

In [ ]:
from transformers import AutoConfig, AutoTokenizer, AutoModelForSequenceClassification

In [ ]:
config = AutoConfig.from_pretrained('beomi/kcbert-base')
tokenizer = AutoTokenizer.from_pretrained('beomi/kcbert-base')
model = AutoModelForSequenceClassification.from_pretrained("beomi/kcbert-base")

In [ ]:
config

In [ ]:
tokenizer

In [ ]:
model

In [ ]:
from datasets import load_metric

# 성능평가지표 
def compute_metrics(eval_pred):
   load_accuracy = load_metric("accuracy")
   load_f1 = load_metric("f1")
  
   logits, labels = eval_pred
   predictions = np.argmax(logits, axis=-1)
   accuracy = load_accuracy.compute(predictions=predictions, references=labels)["accuracy"]
   f1 = load_f1.compute(predictions=predictions, references=labels)["f1"]
   return {"accuracy": accuracy, "f1": f1}

In [ ]:
class NaverReviewDataset(torch.utils.data.Dataset):
    def __init__(self, data, tokenizer):
        self.tokenizer = tokenizer
        self.x = data['review'].values
        self.y = (data['score']>=7).astype(int).values

    def __getitem__(self, idx):
        item = self.tokenizer(self.x[idx], truncation=True)
        item['labels'] = [self.y[idx]]
        return item

    def __len__(self):
        return len(self.x)


train_dataset = NaverReviewDataset(df.iloc[:len(df)//5*4], tokenizer)
test_dataset  = NaverReviewDataset(df.iloc[len(df)//5*4:], tokenizer)

# train_dataset = NaverReviewDataset(df.iloc[:9000], tokenizer)
# test_dataset  = NaverReviewDataset(df.iloc[9000:], tokenizer)

In [ ]:
from transformers import TrainingArguments, Trainer,  DataCollatorWithPadding
 
training_args = TrainingArguments(
   output_dir="finetuning-sentiment",
   learning_rate=2e-5,
   per_device_train_batch_size=5,
   per_device_eval_batch_size=5,
   num_train_epochs=2,
   weight_decay=0.01,
   save_strategy="epoch",
)


data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

trainer = Trainer(
   model=model,
   args=training_args,
   train_dataset=train_dataset,
   eval_dataset=test_dataset,
   tokenizer=tokenizer,
   data_collator=data_collator,
   compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

In [ ]:
trainer.evaluate()

# BERT를 이용한 NER

https://huggingface.co/dslim/bert-base-NER

In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline


In [ ]:


tokenizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER")
model = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER")


In [ ]:
# 간단한 형태의 inference 예제
nlp = pipeline("ner", model=model, tokenizer=tokenizer)
example = "My name is Wolfgang and I live in Berlin"

ner_results = nlp(example)
print(ner_results)

In [ ]:
pd.DataFrame(dataset['train']['ner_tags']).min().min()

In [ ]:
# 학습 예제 - 한국어 NER을 시도해보겠습니다.

tokenizer = AutoTokenizer.from_pretrained('beomi/kcbert-base')
model = AutoModelForTokenClassification.from_pretrained('beomi/kcbert-base')
model.classifier = nn.Linear(model.classifier.in_features, 7) # 0~6까지
model.num_labels = 7

from datasets import load_dataset

dataset = load_dataset("kor_ner")

train_dataset = dataset['train']
val_dataset = dataset['validation']
test_dataset = dataset['test']

In [ ]:
# 데이터셋 예제
train_dataset[0]

In [ ]:
train_dataset[0]

In [ ]:
def preprocess(data):
    input_ids = tokenizer.convert_tokens_to_ids(data['tokens'])
    labels = data['ner_tags']
    if len(input_ids)>tokenizer.model_max_length:
        input_ids = input_ids[:tokenizer.model_max_length]
        labels = labels[:tokenizer.model_max_length]
    input_ids = input_ids + [0] * (tokenizer.model_max_length - len(labels))
    labels = labels + [0] * (tokenizer.model_max_length - len(labels))
    token_type_ids = [0] * len(input_ids)
    attention_mask = [1] * len(data['tokens']) + [0] * (tokenizer.model_max_length - len(data['tokens']))
    return {'input_ids':input_ids, 'token_type_ids':token_type_ids, 'attention_mask':attention_mask, 'labels':labels}

train_dataset = train_dataset.map(preprocess)
val_dataset = val_dataset.map(preprocess)
test_dataset = test_dataset.map(preprocess)

In [ ]:
from transformers import TrainingArguments, Trainer,  DataCollatorForTokenClassification
from datasets import load_metric


metric = load_metric("accuracy")
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=np.reshape(predictions, -1), references=np.reshape(labels,-1))


 
training_args = TrainingArguments(
   output_dir="finetuning-ner",
   learning_rate=2e-5,
   per_device_train_batch_size=16,
   per_device_eval_batch_size=16,
   num_train_epochs=2,
   weight_decay=0.01,
   save_strategy="epoch",
)


data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

trainer = Trainer(
   model=model,
   args=training_args,
   train_dataset=train_dataset,
   eval_dataset=val_dataset,
   tokenizer=tokenizer,
   data_collator=data_collator,
   compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

In [ ]:
trainer.evaluate(eval_dataset=test_dataset)

# KorQuAD 데이터셋을 이용한 QA 모델 만들기

In [ ]:
from datasets import load_dataset
raw_datasets = load_dataset("KETI-AIR/korquad", 'v1.0')

In [ ]:
# 데이터셋마다 구분된 이름이 다를 수 있습니다.
raw_datasets.keys()

In [ ]:
train_dataset = raw_datasets['train']
val_dataset = raw_datasets['dev']

In [ ]:
train_dataset[0]

In [ ]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
tokenizer = AutoTokenizer.from_pretrained('sangrimlee/bert-base-multilingual-cased-korquad')
model = AutoModelForQuestionAnswering.from_pretrained('sangrimlee/bert-base-multilingual-cased-korquad')

In [ ]:
question = train_dataset[0]['question']
context = train_dataset[0]['context']
tokenizer(question, context)

In [ ]:
def preprocess_function(examples):
    inputs = tokenizer(
        examples['question'],
        examples["context"],
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

  

    for i, offset in enumerate([offset_mapping]):
        answer = answers
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label it (0, 0)
        if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    inputs['labels'] = answer
    return inputs

In [ ]:
train_dataset[0]

In [ ]:
preprocess_function(train_dataset[0])

In [ ]:
train_dataset = train_dataset.map(preprocess_function)
val_dataset = val_dataset.map(preprocess_function)

In [ ]:
from transformers import DefaultDataCollator
from datasets import load_metric
accuracy = load_metric('accuracy')
# 성능평가지표 
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    labels = np.concatenate(labels, -1).T
    predictions = np.argmax(logits, axis=-1)
    return accuracy.compute(predictions=predictions.reshape(-1), references=labels.reshape(-1))


data_collator = DefaultDataCollator()




from transformers import TrainingArguments, Trainer
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

In [ ]:
# 이 실습에서 불러온 모델은 이미 korquad 데이터셋에 fine-tuning이 완료된 모델입니다. 학습 없이 evaluation을 해봅시다.
model = AutoModelForQuestionAnswering.from_pretrained('sangrimlee/bert-base-multilingual-cased-korquad')

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset[:10],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)


trainer.evaluate()

## GPT를 이용한 문장 생성

In [ ]:
# pipeline을 이용한 방식
from transformers import pipeline, set_seed
generator = pipeline('text-generation', model='gpt2')

In [ ]:
generator("Hello, I'm a language model,", max_length=50, num_return_sequences=10)


In [ ]:
# pipeline을 사용하지 않는 방식
from transformers import GPT2LMHeadModel, GPT2Tokenizer

model = GPT2LMHeadModel.from_pretrained("gpt2")
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

input_ids = tokenizer.encode("Some text to encode", return_tensors='pt')

generated_text_samples = model.generate(
    input_ids,
    max_length=150,
    num_return_sequences=5,
    no_repeat_ngram_size=2, #2-gram 동어 반복을 피함
    repetition_penalty=1.5,
    top_p=0.92,
    temperature=0.85,
    do_sample=True,
    top_k=125,
    early_stopping=True
)

In [ ]:
for i, beam in enumerate(generated_text_samples):
    print("{}: {}".format(i, tokenizer.decode(beam, skip_special_tokens=True)))
    print()

In [ ]:
# 한국어 GPT

from transformers import PreTrainedTokenizerFast
tokenizer = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2", bos_token='</s>', eos_token='</s>', unk_token='<unk>', pad_token='<pad>', mask_token='<mask>')
tokenizer.tokenize("안녕하세요. 한국어 GPT-2 입니다.😤:)l^o")

In [ ]:
import torch
from transformers import GPT2LMHeadModel

model = GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2')
text = '근육이 커지기 위해서는'
input_ids = tokenizer.encode(text, return_tensors='pt')
gen_ids = model.generate(input_ids,
                           max_length=128,
                           repetition_penalty=2.0,
                           pad_token_id=tokenizer.pad_token_id,
                           eos_token_id=tokenizer.eos_token_id,
                           bos_token_id=tokenizer.bos_token_id,
                           use_cache=True)

In [ ]:
generated = tokenizer.decode(gen_ids[0])
print(generated)

## koBART를 이용한 문단 요약

In [ ]:
from transformers import AutoTokenizer
from transformers import BartForConditionalGeneration

tokenizer = AutoTokenizer.from_pretrained('gogamza/kobart-summarization')
model = BartForConditionalGeneration.from_pretrained('gogamza/kobart-summarization')

In [ ]:
text = "과거를 떠올려보자. 방송을 보던 우리의 모습을. 독보적인 매체는 TV였다. 온 가족이 둘러앉아 TV를 봤다. 간혹 가족들끼리 뉴스와 드라마, 예능 프로그램을 둘러싸고 리모컨 쟁탈전이 벌어지기도  했다. 각자 선호하는 프로그램을 ‘본방’으로 보기 위한 싸움이었다. TV가 한 대인지 두 대인지 여부도 그래서 중요했다. 지금은 어떤가. ‘안방극장’이라는 말은 옛말이 됐다. TV가 없는 집도 많다. 미디어의 혜 택을 누릴 수 있는 방법은 늘어났다. 각자의 방에서 각자의 휴대폰으로, 노트북으로, 태블릿으로 콘텐츠 를 즐긴다."

raw_input_ids = tokenizer.encode(text)
input_ids = [tokenizer.bos_token_id] + raw_input_ids + [tokenizer.eos_token_id]

summary_ids = model.generate(torch.tensor([input_ids]))
tokenizer.decode(summary_ids.squeeze().tolist(), skip_special_tokens=True)